In [1]:
from datasets import load_dataset, load_from_disk
from promptsource import templates

CACHE_DIR = "/share/edc/home/antonis/datasets/huggingface"
import os
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR

/local/home/antonis/.conda/envs/incidental/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_validation.arrow"
dataset = load_from_disk(ds_pth)

In [ ]:
import re
import json

def generate_qa_prompt(example):
    # Extract the tripleset from the input example
    tripleset_match = re.search("tripleset: (.+?) annotations", example)
    if tripleset_match:
        tripleset_str = tripleset_match.group(1)
        tripleset = json.loads(tripleset_str.replace("\'", "\""))
    else:
        return "Error: Could not extract tripleset from example."
    
    # Extract the text (answer) from the input example
    text_match = re.search("text': \[(.+?)]", example)
    if text_match:
        text_str = text_match.group(1)
        # replace escaped single quotes with actual single quotes
        answer = text_str.replace("\\'", "'")
    else:
        return "Error: Could not extract text from example."
    
    # Construct the prompt using the information in the tripleset
    info_list = ["- {} {} {}".format(triple[0], triple[1], triple[2]) for triple in tripleset]
    info_text = "\n".join(info_list)
    prompt = ("Create a concise and grammatically correct sentence that "
              "incorporates the information provided in the triple set. "
              "Please ensure that your sentence naturally integrates this "
              "information:\n{}\nMake sure your sentence reads naturally and is informative."
              .format(info_text))
    
    return {'prompt': prompt, 'answer': answer}


# Example usage:
example = "tripleset: [['Mars Hill College', 'JOINED', '1973'], ['Mars Hill College', 'LOCATION', 'Mars Hill, North Carolina']] annotations: 'source': ['WikiSQL_decl_sents'], 'text': ['A school from Mars Hill, North Carolina, joined in 1973.']"

result = generate_qa_prompt(example)
print("Prompt:", result['prompt'])
print("Answer:", result['answer'])


In [ ]:
import re
import json
import random

def generate_qa_prompt(example):
    # Extract the tripleset from the input example
    tripleset_match = re.search("tripleset: (.+?) annotations", example)
    if tripleset_match:
        tripleset_str = tripleset_match.group(1)
        tripleset = json.loads(tripleset_str.replace("\'", "\""))
    else:
        return "Error: Could not extract tripleset from example."
    
    # Extract the text (answer) from the input example
    text_match = re.search("text': \[(.+?)]", example)
    if text_match:
        text_str = text_match.group(1)
        # replace escaped single quotes with actual single quotes
        answer = text_str.replace("\\'", "'")
    else:
        return "Error: Could not extract text from example."
    
    # Construct the info list
    info_list = ["- {} {} {}".format(triple[0], triple[1], triple[2]) for triple in tripleset]
    info_text = "\n".join(info_list)
    
    # Define 10 different prompt structures
    prompt_structures = [
        "Create a concise and grammatically correct sentence that incorporates the information provided in the triple set. Please ensure that your sentence naturally integrates this information:\n{}\nMake sure your sentence reads naturally and is informative.",
        "Compose a short, grammatically accurate sentence that seamlessly includes the following details:\n{}\nEnsure the sentence flows naturally.",
        "Using the following information, write a clear and concise sentence:\n{}\nThe sentence should be grammatically correct and easy to understand.",
        "Construct a sentence using the information given below. Your sentence should be brief and grammatically correct:\n{}",
        "Your task is to create a sentence that communicates the information below in a clear and natural way:\n{}",
        "Write a grammatical sentence that integrates the following information in a coherent manner:\n{}",
        "Please formulate a sentence using the information provided below. Ensure it is grammatically correct and makes logical sense:\n{}",
        "Using the data points below, create a single sentence that is grammatically correct and effectively communicates the information:\n{}",
        "Construct a grammatically accurate and informative sentence using the details given below:\n{}",
        "Combine the information provided into a single, grammatically correct sentence that reads naturally:\n{}"
    ]
    
    # Randomly select a prompt structure
    prompt = random.choice(prompt_structures).format(info_text)
    
    return {'prompt': prompt, 'answer': answer}


# Example usage:
example = "tripleset: [['Mars Hill College', 'JOINED', '1973'], ['Mars Hill College', 'LOCATION', 'Mars Hill, North Carolina']] annotations: 'source': ['WikiSQL_decl_sents'], 'text': ['A school from Mars Hill, North Carolina, joined in 1973.']"

result = generate_qa_prompt(example)
print("Prompt:", result['prompt'])
print("Answer:", result['answer'])